In [ ]:
# !pip install --pre pycaret[full]
# !pip install packaging==21.3

In [ ]:
# !pip install --upgrade packaging
# !pip install --upgrade pycaret

In [ ]:
import pandas as pd
import numpy as np
import warnings
from pycaret.time_series import *
from sktime.forecasting.model_selection import SlidingWindowSplitter
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv("./after_clean/cleaned_Songkhla.csv")
df["DATETIMEDATA"] = pd.to_datetime(df["DATE"] + ' ' + df["TIME"], format="%Y-%m-%d %H:%M:%S")

df = df[['DATETIMEDATA','PM25']]
df = df.set_index('DATETIMEDATA')
df

In [ ]:
exp = TSForecastingExperiment()
exp.setup(data = df, target='PM25' ,coverage=0.90, fold_strategy=SlidingWindowSplitter(fh=np.arange(1,25), window_length=130, step_length=130)) 

In [ ]:
exp.compare_models(sort='rmse')

In [ ]:
df1 = pd.read_csv("./after_clean/cleaned_Songkhla.csv")
df2 = pd.read_csv("./after_clean/cleaned_Mueang_Yala.csv")
df3 = pd.read_csv("./after_clean/cleaned_Betong_Yala.csv")
df4 = pd.read_csv("./after_clean/cleaned_Nakhon_Si_Thammarat.csv")
df5 = pd.read_csv("./after_clean/cleaned_Narathiwat.csv")
df6 = pd.read_csv("./after_clean/cleaned_Phuket.csv")
df7 = pd.read_csv("./after_clean/cleaned_Satun.csv")
df8 = pd.read_csv("./after_clean/cleaned_Surat_Thani.csv")
df9 = pd.read_csv("./after_clean/cleaned_Trang.csv")

In [ ]:
df_list = [df1, df2, df3, df4, df5, df6, df7, df8, df9]

Province = ["Songkhla","Mueang_Yala","Betong_Yala",
            "Nakhon_Si_Thammarat","Narathiwat","Phuket",
            "Satun","Surat_Thani","Trang"]

In [ ]:
count = 0
for df in df_list:
    df["DATETIMEDATA"] = pd.to_datetime(df["DATE"] + ' ' + df["TIME"], format="%Y-%m-%d %H:%M:%S")
    df["DATETIMEDATA"] = pd.to_datetime(df["DATETIMEDATA"], format="%Y-%m-%d %H:%M:%S")
    df.sort_values("DATETIMEDATA", inplace=True)
    df = df[['DATETIMEDATA', 'PM25']]
    df.set_index('DATETIMEDATA', inplace=True)
    
    exp = TSForecastingExperiment()
    exp.setup(data=df, target='PM25', coverage=0.90, fold_strategy=SlidingWindowSplitter(fh=np.arange(1, 25), window_length=130, step_length=130)) 
    
    llar_cds_dt = exp.create_model('llar_cds_dt')
    tuned_llar_cds_dt = exp.tune_model(llar_cds_dt)
    holdout_pred = exp.predict_model(tuned_llar_cds_dt)
    final_model = exp.finalize_model(tuned_llar_cds_dt)

    unseen_predictions = exp.predict_model(final_model, fh=168)
    DATETIMEDATA = pd.date_range(start='2024-03-13', end='2024-03-19 23:00:00', freq='H')
    unseen_predictions['DATETIMEDATA'] = DATETIMEDATA

    unseen_predictions['DATE'] = unseen_predictions['DATETIMEDATA'].dt.date
    unseen_predictions['TIME'] = unseen_predictions['DATETIMEDATA'].dt.time

    unseen_predictions.drop(columns=['DATETIMEDATA'], inplace=True)

    cols = list(unseen_predictions.columns)
    cols.remove('y_pred')  
    cols.insert(2, 'y_pred')  
    unseen_predictions = unseen_predictions[cols]
    unseen_predictions = unseen_predictions.rename(columns={'y_pred': 'PM25'})

    unseen_predictions.to_csv(f'./PM25_predictions/PM25_{Province[count]}.csv', index=False)
    count += 1